In [2]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType


In [3]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .config("spark.driver.memory", "8g") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/16 18:54:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Datasets
- user_logs.csv — contain data from 2015/01/01 - 2017/02/28
- user_logs_v2.csv — contain data from 2017/03/01 - 2017/03/31
- transactions.csv — (transaction_date) 2015/01/01 - 2017/02/28
- transactions_v2.csv — (transaction_date) 2015/01/01 - 2017/03/31 (has lesser rows tho ??)
- members_v3.csv — 6mill members
- train.csv — contains user ids and whether they have churned.
- train_v2.csv — contains the churn data for March, 2017.


# User Logs

In [15]:

df = (spark.read
      .option("header", True)
      .option("inferSchema", True)
      .parquet("datamart/bronze/user_logs/year=2015/month=01"))


In [16]:
df.cache()

DataFrame[msno: string, date: int, num_25: int, num_50: int, num_75: int, num_985: int, num_100: int, num_unq: int, total_secs: double]

In [17]:
df.count()

12897706

In [18]:
df.show()

+--------------------+--------+------+------+------+-------+-------+-------+----------+
|                msno|    date|num_25|num_50|num_75|num_985|num_100|num_unq|total_secs|
+--------------------+--------+------+------+------+-------+-------+-------+----------+
|C4StQg63rttGAOqto...|20150120|     2|     2|     1|      0|      6|      9|  1699.446|
|8lZ3LnlnX1kWyKkGt...|20150109|     5|     2|     1|      0|     82|     55| 17811.605|
|GCqC99i1NrtvDGMkx...|20150129|     2|     1|     0|      1|    129|      4| 34703.904|
|Dnpj5VLwwN5mUR8HA...|20150128|     0|     0|     0|      1|      7|      7|  2162.177|
|l1PoAgkJoUkdBz9aH...|20150112|     2|     0|     0|      0|     34|     27|  9284.661|
|xrIHhq+6/NnEwklQp...|20150120|     4|     1|     1|      1|    161|    115| 40552.138|
|9iUqNqp4tOo/Gu+dk...|20150115|    20|     6|     1|      5|     30|     21|  9404.582|
|fkYsxuAJ0fyDu8wqj...|20150107|     2|     0|     0|      0|     36|     32|  8196.252|
|vEB1w4iqIf3SvU37X...|20150106| 

In [20]:
# Check for duplicate user logs for the same day
duplicate_count = df.groupBy("msno", "date").count().where("count > 1").count()

if duplicate_count > 0:
    print(f"Found {duplicate_count} instances of users with multiple logs on the same day.")
else:
    print("✅ No duplicate user-date entries found.")

✅ No duplicate user-date entries found.


In [21]:
from pyspark.sql.functions import col

# Check for negative values in any of the numeric columns
negative_counts = df.where(
    (col("num_25") < 0) |
    (col("num_50") < 0) |
    (col("num_75") < 0) |
    (col("num_985") < 0) |
    (col("num_100") < 0) |
    (col("num_unq") < 0) |
    (col("total_secs") < 0)
).count()

print(f"Found {negative_counts} rows with negative numeric values.")

# Check for listening times that are impossibly long (more than 24 hours)
impossible_time_count = df.where(col("total_secs") > 86400).count()
print(f"Found {impossible_time_count} rows where total_secs exceeds 24 hours.")

Found 3 rows with negative numeric values.


[Stage 35:>                                                       (0 + 12) / 13]

Found 7226 rows where total_secs exceeds 24 hours.


In [23]:
# Get a statistical summary to identify outliers
df.select("num_25", "num_100", "num_unq", "total_secs").describe().show()

[Stage 41:============>                                           (3 + 10) / 13]

+-------+------------------+------------------+-----------------+--------------------+
|summary|            num_25|           num_100|          num_unq|          total_secs|
+-------+------------------+------------------+-----------------+--------------------+
|  count|          12897706|          12897706|         12897706|            12897706|
|   mean| 6.098357645925562|31.652541157318982|29.88532697209876|-2.14534248473938...|
| stddev|13.421142334833364|46.369546292679864|34.34555369998261|4.448300153104476E12|
|    min|                 0|                 0|                1|-9.22337203685156...|
|    max|             11328|             34696|             1694|    1.420659527338E9|
+-------+------------------+------------------+-----------------+--------------------+



In [24]:
from pyspark.sql.functions import col

# This is a rough approximation for sanity checking
df_with_approx_secs = df.withColumn("approx_secs",
    col("num_25") * 25 +
    col("num_50") * 50 +
    col("num_75") * 75 +
    col("num_100") * 240  # Assuming an average song length of 4 minutes for num_100
)

# Show rows where the reported total_secs is drastically different from our approximation
# This could indicate data quality issues.
df_with_approx_secs.select("msno", "date", "total_secs", "approx_secs").show()

+--------------------+--------+----------+-----------+
|                msno|    date|total_secs|approx_secs|
+--------------------+--------+----------+-----------+
|C4StQg63rttGAOqto...|20150120|  1699.446|       1665|
|8lZ3LnlnX1kWyKkGt...|20150109| 17811.605|      19980|
|GCqC99i1NrtvDGMkx...|20150129| 34703.904|      31060|
|Dnpj5VLwwN5mUR8HA...|20150128|  2162.177|       1680|
|l1PoAgkJoUkdBz9aH...|20150112|  9284.661|       8210|
|xrIHhq+6/NnEwklQp...|20150120| 40552.138|      38865|
|9iUqNqp4tOo/Gu+dk...|20150115|  9404.582|       8075|
|fkYsxuAJ0fyDu8wqj...|20150107|  8196.252|       8690|
|vEB1w4iqIf3SvU37X...|20150106|  2326.174|       2185|
|ccqIvziyLZs/Ivbob...|20150110|  1605.832|       1465|
|xKKR50oC2MlZ0zd+o...|20150105|    70.019|         75|
|TUWjZbEJkEvyq8rv4...|20150131|    20.651|         25|
|k6F2zRxgWsYSNWHK2...|20150117|  3155.356|       2525|
|dmvdNxpnayMipYpp0...|20150123|  2821.996|       2070|
|7rP5PTRRZkv88mQaW...|20150129|   562.586|        480|
|bYBeBROvV

In [7]:
df2 = (spark.read
      .option("header", True)
      .option("inferSchema", True)
      .csv("data/user_logs_v2.csv"))


In [8]:
df2.count()

18396362

In [52]:
from pyspark.sql import functions as F

df2.select(
    F.min("date").alias("start_date"),
    F.max("date").alias("end_date")
).show()

[Stage 93:>                                                       (0 + 12) / 12]

+----------+--------+
|start_date|end_date|
+----------+--------+
|  20170301|20170331|
+----------+--------+



# Transactions

In [42]:
df_transactions = (spark.read
      .option("header", True)
      .option("inferSchema", True)
      .csv("data/transactions.csv"))

In [43]:
df_transactions.show()

+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+
|                msno|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+
|YyO+tlZtAXYXoZhNr...|               41|               30|            129|               129|            1|        20150930|              20151101|        0|
|AZtu6Wl0gPojrEQYB...|               41|               30|            149|               149|            1|        20150930|              20151031|        0|
|UkDFI97Qb6+s2LWci...|               41|               30|            129|               129|            1|        20150930|              20160427|        0|
|M1C56ijxozNaGD0t2...|               39|            

In [44]:
df_transactions.count()

21547746

In [55]:
from pyspark.sql import functions as F

df_transactions.select(
    F.min("transaction_date").alias("start_date"),
    F.max("transaction_date").alias("end_date")
).show()


[Stage 102:==================================================>    (12 + 1) / 13]

+----------+--------+
|start_date|end_date|
+----------+--------+
|  20150101|20170228|
+----------+--------+



In [46]:
df_transactions_v2 = (spark.read
      .option("header", True)
      .option("inferSchema", True)
      .csv("data/transactions_v2.csv"))

In [57]:
df_transactions_v2.count()

1431009

In [56]:
from pyspark.sql import functions as F

df_transactions_v2.select(
    F.min("transaction_date").alias("start_date"),
    F.max("transaction_date").alias("end_date")
).show()

+----------+--------+
|start_date|end_date|
+----------+--------+
|  20150101|20170331|
+----------+--------+



# Members

In [58]:
df_members = (spark.read
      .option("header", True)
      .option("inferSchema", True)
      .csv("data/members_v3.csv"))

In [59]:
df_members.show()

+--------------------+----+---+------+--------------+----------------------+
|                msno|city| bd|gender|registered_via|registration_init_time|
+--------------------+----+---+------+--------------+----------------------+
|Rb9UwLQTrxzBVwCB6...|   1|  0|  NULL|            11|              20110911|
|+tJonkh+O1CA796Fm...|   1|  0|  NULL|             7|              20110914|
|cV358ssn7a0f7jZOw...|   1|  0|  NULL|            11|              20110915|
|9bzDeJP6sQodK73K5...|   1|  0|  NULL|            11|              20110915|
|WFLY3s7z4EZsieHCt...|   6| 32|female|             9|              20110915|
|yLkV2gbZ4GLFwqTOX...|   4| 30|  male|             9|              20110916|
|jNCGK78YkTyId3H3w...|   1|  0|  NULL|             7|              20110916|
|WH5Jq4mgtfUFXh2yz...|   5| 34|  male|             9|              20110916|
|tKmbR4X5VXjHmxERr...|   5| 19|  male|             9|              20110917|
|I0yFvqMoNkM8ZNHb6...|  13| 63|  male|             9|              20110918|

In [60]:
df_members.count()

6769473

# Train

In [61]:
df_train = (spark.read
      .option("header", True)
      .option("inferSchema", True)
      .csv("data/train.csv"))

In [62]:
df_train.show()

+--------------------+--------+
|                msno|is_churn|
+--------------------+--------+
|waLDQMmcOu2jLDaV1...|       1|
|QA7uiXy8vIbUSPOkC...|       1|
|fGwBva6hikQmTJzrb...|       1|
|mT5V8rEpa+8wuqi6x...|       1|
|XaPhtGLk/5UvvOYHc...|       1|
|GBy8qSz16X5iYWD+3...|       1|
|lYLh7TdkWpIoQs3i3...|       1|
|T0FF6lumjKcqEO0O+...|       1|
|Nb1ZGEmagQeba5E+n...|       1|
|MkuWz0Nq6/Oq5fKqR...|       1|
|I8dFN2EjFN1mt4Xel...|       1|
|0Ip2rzeoa44alqEw3...|       1|
|piVhWxrWDmiNQFY6x...|       1|
|wEUOkYvyz3xTOx2p9...|       1|
|xt4EjWRyXBMgEgKBJ...|       1|
|QS3ob4zLlWcWzBIlb...|       1|
|9iW/UpqRoviya9CQh...|       1|
|d7QVMhAzjj4yc1Ojj...|       1|
|uV7rJjHPrpNssDMmY...|       1|
|TZxhkfZ9NwxqnUrNs...|       1|
+--------------------+--------+
only showing top 20 rows



In [63]:
df_train.count()

992931

In [64]:
df_train_v2 = (spark.read
      .option("header", True)
      .option("inferSchema", True)
      .csv("data/train_v2.csv"))

In [66]:
df_train_v2.count()

970960

In [ ]:
df_train_v2.show()